# **Dependencies**

This is easier to run inside Google Colab, but it can also be ran in your own machine. In that case, a GPU is recommended, otherwise this will take ages to run.

Depending on the case, run the appropriate cell bellow to install the required dependencies:

In [0]:
# If you are NOT in Google Colab, run this:

!pip install PyDrive tensorflow matplotlib numpy scikit-image

In [0]:
# If you are insode Google Colab, run this instead

!pip install PyDrive
%tensorflow_version 2.x

In [0]:
import time

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from skimage.transform import resize
from tensorflow.keras import layers

from IPython import display

# stuff used to get the file from google drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# which version of tensorflow are we using? and are we using a gpu?
print('Tensorflow version:', tf.__version__)

tf_device = tf.test.gpu_device_name()
if 'GPU' not in tf_device:
    print('GPU not found :(')
else:
    print('Using GPU: {}'.format(tf_device))

# **Getting the data**

To run this example, we need to get a file which contains the example images. This file can be downloaded from:
https://drive.google.com/open?id=1JWwYHf-mTtq1JUWMfrqFNtjm4yOvz1bc

If you are running this in yuor machine, download the file and place it in the same folder as the notebook. And don't run this cell.

If you are running this inside Google Colab, run this cell and follow the instructions to get the file into the Colab runtime.


In [0]:
# if we are running inside Google Colab, automatically get the images file from
# Google Drive:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1JWwYHf-mTtq1JUWMfrqFNtjm4yOvz1bc'}) 
downloaded.GetContentFile('profes_images_array.pkl')

# **Reading the data**

The pickle file contains images, that are 32x32 pixels, with 3 color channels (red, green, blue):


In [0]:
real_images = np.load('profes_images_array.pkl')
real_images.shape

In [0]:
# sample pixel
real_images[500][0][0]

In [0]:
def sample(images, sample_size):
    """
    Make a random sample of a set of images.
    """
    return images[np.random.randint(len(images), size=sample_size)]


def show_images_grid(images, normalized=True, grid_size=3, fig_size=10, 
                     file_name=None):
    """
    Given a set of images, show them together and optionally save the 
    collage to a file.
    """
    assert len(images) <= (grid_size ** 2)
    fig = plt.figure(figsize=(fig_size, fig_size))

    for i, image in enumerate(images):
        plt.subplot(grid_size, grid_size, i+1)
        plt.axis('off')
        if normalized:
            plt.imshow((image / 2) + 0.5)
        else:
            plt.imshow(image)

    if file_name is not None:
        plt.savefig(file_name)

    plt.show()

In [0]:
show_images_grid(sample(real_images, 9), normalized=False)

In [0]:
# normalize the images so the numbers go between -1 and 1
processed_real_images = ((real_images - 127.5) / 127.5)

# resize the images, so we can train faster during the demo
processed_real_images = np.array([resize(image, (16, 16, 3))
                                  for image in processed_real_images])
# and use a smaller data type
processed_real_images = processed_real_images.astype('float32')

processed_real_images.shape

In [0]:
# sample normalized pixel (same pixel we saw before)
processed_real_images[500][0][0]

In [0]:
show_images_grid(sample(processed_real_images, 9))

In [0]:
# build a TF dataset, increasing the number of images (repeating them), and 
# grouping them in batches for easier training
BATCH_SIZE = 256
real_images_dataset = tf.data.Dataset.from_tensor_slices(processed_real_images)\
                                     .shuffle(100000)\
                                     .batch(BATCH_SIZE)

# **The neural networks**

Here we create the two networks, that will be trained together.

In [0]:
# this is the network that will generate images
# - it will receive random noise as input
# - the ouput will be a normalized image with shape 32x32x3

generator = tf.keras.Sequential([
    layers.Dense(4 * 4 * 256, use_bias=False, input_shape=(100,)),
    layers.LeakyReLU(),

    layers.Reshape((4, 4, 256)),

    layers.Conv2DTranspose(128, (4, 4), strides=1, padding='same',
                           use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(),

    layers.Conv2DTranspose(64, (4, 4), strides=2, padding='same',
                           use_bias=False),
    layers.BatchNormalization(),
    layers.LeakyReLU(),

    layers.Conv2DTranspose(3, (4, 4), strides=2, padding='same', use_bias=False,
                           activation='tanh'),
])

assert generator.output_shape == (None, 16, 16, 3)

In [0]:
# lets test it!
# give random noise to the generator, and see what images it produces!
# (of course, it's still quite dumb)

noise = tf.random.normal([9, 100])
generated_images = generator(noise, training=False)

show_images_grid(generated_images)

In [0]:
# this is the network that will try to guess which image is real and which image
# is generated (fake)
# - it will receive a normalized image as input (32x32x3)
# - the ouput is a single number, that we expect it to say if the image is fake
#   (negative) or real (positive).

discriminator = tf.keras.Sequential([
    layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                    input_shape=[16, 16, 3]),
    layers.LeakyReLU(),
    layers.Dropout(0.3),

    layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    layers.LeakyReLU(),
    layers.Dropout(0.3),

    layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    layers.LeakyReLU(),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(32),
    layers.Dense(1),
])

In [0]:
# lets test it!
# give it a few fake images, and see what it says about them
# (of course, it's still quite dumb)

discriminator(generated_images)

In [0]:
# give it a few real images, and see what it says about them
# (still quite dumb)

discriminator(sample(processed_real_images, 9))

# **How will the networks learn**

To train these networks, we need some kind of loss ("error") function that describes how much did they fail in their tasks. This will be used during training, to help them learn to do their work better.

In [0]:
 # binary cross entropy is a function that tells lets us measure errors between
 # sets of numbers that go between 0 and 1

 # TODO sacar from_logits, agregar activacion a la salida de discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
# evaluating the discriminator net:
# we gave the net 2 sets of images: reals and fakes. We expect it to answer "1"
# in all the real images, and "0" in all the fake images
# the "error" of the discriminator net is then, how different its predictions
# have been, compared to those expected outputs

def discriminator_loss(real_images_predictions, fake_images_predictions):
    # cross entropy between values predicted for real images, and all 1s
    real_loss = cross_entropy(tf.ones_like(real_images_predictions), 
                              real_images_predictions)
    # cross entropy between values predicted for fake images, and all 0s
    fake_loss = cross_entropy(tf.zeros_like(fake_images_predictions), 
                              fake_images_predictions)
    # sum the errors of the real and fake images
    return real_loss + fake_loss

In [0]:
# evaluating the generator net:
# given a set of fake images it produced, that we then gave to the 
# discriminator, a perfect generator should have created images in a way that
# all were classified as reals
# So the "error" of the generator, is how different the fake images have been 
# classified by the discriminator, in comparisson to all 1s (all "real")

def generator_loss(fake_images_predictions):
    # entropy between values predicted for fake images, and all 1s
    return cross_entropy(tf.ones_like(fake_images_predictions), 
                         fake_images_predictions)

In [0]:
# both nets will use gradient descent to train their weights, here we create 
# their GD optimizers (Adam variant) with a learning rate of 0.001

discriminator_optimizer = tf.keras.optimizers.Adam(0.0003)
generator_optimizer = tf.keras.optimizers.Adam(0.0003)

# **Traning logic**

And now we define the general training logic.

The training will be done in steps. At each step, we take a batch of real images, and generate another batch of fake images using the generator network.

We then give the two batches to the discriminator network, and measure the losses for both networks.
After this, we tell the networks to learn from their errors (using the adam optimizers).

And finally at each step, we also generage a batch of fake images using always the same random noise, so we can visualize how the learning evolves, how the same random nose input is creating better and better images on each training step.

In [0]:
@tf.function
def train_step(real_images):
    """
    A training step, using a batch of real images.
    """
    noise = tf.random.normal([BATCH_SIZE, 100])

    with tf.GradientTape() as generator_tape,\
         tf.GradientTape() as discriminator_tape:
        # generate a batck of fake images too
        fake_images = generator(noise, training=True)

        # ask the discriminator to analyze both sets of images
        real_images_predictions = discriminator(real_images, training=True)
        fake_images_predictions = discriminator(fake_images, training=True)

        # calculate the error of both networks
        current_discriminator_loss = discriminator_loss(
            real_images_predictions, fake_images_predictions
        )
        current_generator_loss = generator_loss(fake_images_predictions)

    # calculate the changes needed for both networks to get better
    generator_gradients = generator_tape.gradient(
        current_generator_loss, generator.trainable_variables
    )
    discriminator_gradients = discriminator_tape.gradient(
        current_discriminator_loss, discriminator.trainable_variables
    )

    # apply the changes, so they become better at their tasks
    generator_optimizer.apply_gradients(
        zip(generator_gradients, generator.trainable_variables)
    )
    discriminator_optimizer.apply_gradients(
        zip(discriminator_gradients, discriminator.trainable_variables)
    )

In [0]:
def train(dataset, epochs, evolution_noise=None):
    """
    Train both models at the same time! Execute the train_step function a number
    of epochs, plus generate the sample images at each epoch.
    If evolution_noise is specified, that same noise is used to generate images
    on each epoch.
    """
    for epoch in range(epochs):
        start = time.time()

        # go through the whole dataset in batches, learning at each batch of
        # images
        for real_images_batch in dataset:
            train_step(real_images_batch)

        # generate the visualization images
        if evolution_noise is None:
            noise = tf.random.normal([9, 100])
        else:
            noise = evolution_noise
        generated_images = generator(noise, training=False)
        show_images_grid(generated_images, fig_size=6,
                         file_name='./evolution/{}.png'.format(start))

        print('Epoch', epoch + 1,
              'finished in', time.time() - start, 
              'seconds')
        
        # clear the notebook output
        display.clear_output(wait=True)

# **Now... train!!**

Finally, we can put the networks to work and learn.

In [0]:
!mkdir ./evolution -p
!rm ./evolution/*.png

In [0]:
train(real_images_dataset, 1500)

# **Test the trained networks**

Now that we have both networks trained, test them by generating images and classifying them.

In [0]:
# lets test the trained generator!
# give random noise to the generator, and see what image it produces!

noise = tf.random.normal([9, 100])
generated_images = generator(noise, training=False)
show_images_grid(generated_images)

In [0]:
# lets test the trained discriminator!
# give it a few fake images, and see what it says about them

discriminator(generated_images)

In [0]:
# and give it a few real images, and see what it says about them

discriminator(sample(processed_real_images, 9))